### 还有子集问题，多维数组初始化问题。



## 1. 0-1背包问题

给你一个可装载重量为 W 的背包和 N 个物品，每个物品有重量和价值两个属性。
其中第 i 个物品的重量为 wt[i]，价值为 val[i]，现在让你用这个背包装物品，
最多能装的价值是多少？

状态转移方程为：

$$F[i, v]=\max \left\{F[i-1, v], F\left[i-1, v-C_{i}\right]+W_{i}\right\}$$

取决于你第i个物品拿不拿。

In [6]:
def knapsack01(values, weights, capacity):
    n = len(values)
    dp = [[0 for _ in range(capacity+1)] for _ in range(n+1)]
    for i in range(n+1):
        for j in range(weights[i-1], capacity+1):
            dp[i][j] = max(dp[i-1][j], dp[i-1][j-weights[i-1]] + values[i-1])
    return dp[n-1][capacity]

In [7]:
print(knapsack01([40, 100, 50, 60],  [20, 10, 40, 30], 60))


200


因为dp[i][j]只用到了dp[i-1]相关的东西，我们可以只保留一行。如果倒着运算，dp[j]
用到的是dp[j-weights[i-1]]，这个还没有被更新，是之前i-1时候算的值。所以



In [17]:
def knapsack01_compress(values, weights, capacity):
    n = len(values)
    dp = [0 for _ in range(capacity+1)]
    for i in range(n+1):
        for j in range(capacity, weights[i-1]-1, -1):
            dp[j] = max(dp[j], dp[j-weights[i-1]] + values[i-1])
    return dp[capacity]

In [18]:
print(knapsack01_compress([40, 100, 50, 60],  [20, 10, 40, 30], 60))

200
